In [12]:
from hydra_gnn.utils import PROJECT_DIR, MP3D_BENCHMARK_DIR, HYDRA_TRAJ_DIR
from hydra_gnn.mp3d_utils import generate_mp3d_split, read_mp3d_split
from hydra_gnn.mp3d_dataset import Hydra_mp3d_dataset
from hydra_gnn.base_training_job import BaseTrainingJob
import random
import os
import shutil
import argparse
import pickle
import yaml
import pandas as pd

In [3]:
import torch
torch.cuda.is_available()

True

In [15]:
# Parameters
config = dict()
config['data'] = {
    'file_path':"output/preprocessed_mp3d/data_test.pkl",
    'type': 'heterogeneous'
}
config['run_control'] = {
    'num_runs': 5,
    'early_stop_window': 300
}
config['network'] = {
    'conv_block': 'GAT_edge',
    'dropout': 0.25,
    'GAT_hidden_dims': [4],
    'GAT_heads': [4, 4],
    'GAT_concats': [True, False]
}
config['optimization'] = {
    'lr': 0.001,
    'num_epochs': 1000,
    'weight_decay': 0.0,
    'batch_size': 1024
}
config['logger'] = {
    'output_dir': "output/log_test"
}

In [7]:
dataset_path = os.path.join(PROJECT_DIR, config['data']['file_path'])
output_dir = os.path.join(PROJECT_DIR, config['logger']['output_dir'])
num_runs = config['run_control']['num_runs']
network_type = config['data']['type']
early_stop_window = config['run_control']['early_stop_window']
network_params = config['network']
optimization_params = config['optimization']

print(network_type)
print(dataset_path)

neural_tree
/home/gridsan/siyi/research_code/Hydra-GNN/output/preprocessed_mp3d/data_test.pkl


## Load dataset

In [9]:
split_dict = read_mp3d_split(MP3D_BENCHMARK_DIR)
dataset_dict = {'train': Hydra_mp3d_dataset('train'),
                'val': Hydra_mp3d_dataset('val'),
                'test': Hydra_mp3d_dataset('test')}

with open(dataset_path, 'rb') as input_file:
    data_list = pickle.load(input_file)
if network_type == 'homogeneous':
    [data.to_homoegeneous() for data in data_list]
if config['network']['conv_block'] == 'GAT_edge':
    [data.compute_relative_pos() for data in data_list]

for data in data_list:
    if data.get_data_info()['scene_id'] in split_dict['scenes_train']:
        dataset_dict['train'].add_data(data)
    elif data.get_data_info()['scene_id'] in split_dict['scenes_val']:
        dataset_dict['val'].add_data(data)
    elif data.get_data_info()['scene_id'] in split_dict['scenes_test']:
        dataset_dict['test'].add_data(data)
    else:
        raise RuntimeError(f"Founnd invalid scene id in input data file {dataset_path}.")
print(f"  training: {dataset_dict['train'].num_scenes()} scenes {len(dataset_dict['train'])} graphs\n"
      f"  validation: {dataset_dict['val'].num_scenes()} scenes {len(dataset_dict['val'])} graphs\n"
      f"  test: {dataset_dict['test'].num_scenes()} scenes {len(dataset_dict['test'])} graphs")

  training: 61 scenes 4176 graphs
  validation: 11 scenes 731 graphs
  test: 18 scenes 1207 graphs


## Run experiment

In [17]:
experiment_output_dir = os.path.join(PROJECT_DIR, config['logger']['output_dir'])
print(f"output direcotry: {experiment_output_dir}")
if os.path.exists(experiment_output_dir):
    input("Output directory exists. Existing contents might be over-written. Press any key to proceed...")
else:
    os.mkdir(experiment_output_dir)

test_accuracy_list = []
val_accuracy_list = []
training_time_list = []
training_epoch_list = []
test_time_list = []
for j in range(config['run_control']['num_runs']):
    train_job = BaseTrainingJob(network_type=config['data']['type'],
                                dataset_dict=dataset_dict, 
                                network_params=config['network'])
    model, best_acc, info = train_job.train(experiment_output_dir + '/' + str(j), 
                                    optimization_params=config['optimization'],
                                    early_stop_window=config['run_control']['early_stop_window'], 
                                    verbose=True)

    val_accuracy_list.append(best_acc[0] * 100)
    test_accuracy_list.append(best_acc[1] * 100)
    training_time_list.append(info['training_time'])
    training_epoch_list.append(info['num_epochs'])
    test_time_list.append(info['test_time'])

output direcotry: /home/gridsan/siyi/research_code/Hydra-GNN/output/log_test
Epoch 009. Loss: 2.6551. Train accuracy: 0.2519. Validation accuracy: 0.1835.
Epoch 019. Loss: 2.3270. Train accuracy: 0.3340. Validation accuracy: 0.2651.
Epoch 029. Loss: 2.0135. Train accuracy: 0.4393. Validation accuracy: 0.3307.
Epoch 039. Loss: 1.8201. Train accuracy: 0.4982. Validation accuracy: 0.3463.
Epoch 049. Loss: 1.6870. Train accuracy: 0.5400. Validation accuracy: 0.3765.
Epoch 059. Loss: 1.6090. Train accuracy: 0.5567. Validation accuracy: 0.3849.
Epoch 069. Loss: 1.5536. Train accuracy: 0.5752. Validation accuracy: 0.3816.
Epoch 079. Loss: 1.5086. Train accuracy: 0.5834. Validation accuracy: 0.3802.
Epoch 089. Loss: 1.4794. Train accuracy: 0.5974. Validation accuracy: 0.3758.
Epoch 099. Loss: 1.4417. Train accuracy: 0.5974. Validation accuracy: 0.3791.
Epoch 109. Loss: 1.4033. Train accuracy: 0.6108. Validation accuracy: 0.3733.
Epoch 119. Loss: 1.3902. Train accuracy: 0.6202. Validation accur

ValueError: not enough values to unpack (expected 3, got 2)